# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [165]:
import pandas as pd
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

In [147]:
df=pd.read_csv('flights.csv')

In [148]:
df.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,0,2019-09-25,WN,WN,WN,954,WN,N232WN,954,14193,PNS,"Pensacola, FL",10693,BNA,"Nashville, TN",1930,1922.0,-8.0,11.0,1933.0,2031.0,4.0,2045,2035.0,-10.0,0.0,NaN,0.0,N,75.0,73.0,58.0,1.0,391.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-05-07,UA,UA,UA,740,UA,N843UA,740,11292,DEN,"Denver, CO",13495,MSY,"New Orleans, LA",949,944.0,-5.0,12.0,956.0,1258.0,2.0,1322,1300.0,-22.0,0.0,NaN,0.0,N,153.0,136.0,122.0,1.0,1062.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-07-02,AS,AS,AS,1097,AS,N853VA,1097,12264,IAD,"Washington, DC",12892,LAX,"Los Angeles, CA",1613,1605.0,-8.0,15.0,1620.0,1807.0,6.0,1852,1813.0,-39.0,0.0,NaN,0.0,N,339.0,308.0,287.0,1.0,2288.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2019-07-02,AA,AA,AA,1194,AA,N934NN,1194,13303,MIA,"Miami, FL",13204,MCO,"Orlando, FL",1730,1732.0,2.0,12.0,1744.0,1819.0,13.0,1847,1832.0,-15.0,0.0,NaN,0.0,N,77.0,60.0,35.0,1.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2019-06-28,WN,WN,WN,2543,WN,N7864B,2543,11540,ELP,"El Paso, TX",14107,PHX,"Phoenix, AZ",2100,2058.0,-2.0,7.0,2105.0,2100.0,15.0,2120,2115.0,-5.0,0.0,NaN,0.0,N,80.0,77.0,55.0,1.0,347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
df_flights=df[['fl_date', 'air_time','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier','mkt_carrier_fl_num',
                       'op_unique_carrier', 'tail_num','op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
                       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time','dep_delay','crs_arr_time', 'dup', 'crs_elapsed_time',
                       'flights', 'distance','arr_delay','cancelled','diverted']]

In [150]:
df_flights.isnull().sum()

fl_date                 0
air_time              949
mkt_unique_carrier      0
branded_code_share      0
mkt_carrier             0
mkt_carrier_fl_num      0
op_unique_carrier       0
tail_num              156
op_carrier_fl_num       0
origin_airport_id       0
origin                  0
origin_city_name        0
dest_airport_id         0
dest                    0
dest_city_name          0
crs_dep_time            0
dep_delay             786
crs_arr_time            0
dup                     0
crs_elapsed_time        0
flights                 0
distance                0
arr_delay             942
cancelled               0
diverted                0
dtype: int64

In [151]:
df_flights=df_flights.dropna()

## Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [152]:
# Converting date and time columns
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'], errors='coerce')
df_flights['month'] = df_flights['fl_date'].dt.month
df_flights['day_of_week'] = df_flights['fl_date'].dt.dayofweek
df_flights['day_of_month'] = df_flights['fl_date'].dt.day
df_flights['year'] = df_flights['fl_date'].dt.year

In [153]:
# Creating columns for departure and arrival hours
df_flights['crs_dep_time'] = df_flights['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))
df_flights['crs_arr_time'] = df_flights['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x))
df_flights['dep_hour']=df_flights['crs_dep_time'].str[:2]
df_flights['arr_hour']=df_flights['crs_arr_time'].str[:2]

In [154]:
# Dividing the months into four seasons
df_flights['seasons']='spring'
df_flights.loc[df_flights['month'].between(6, 8), 'seasons']= 'summer'
df_flights.loc[df_flights['month'].between(9, 11), 'seasons']= 'autum'
df_flights.loc[df_flights['month'].between(12, 2), 'seasons']= 'winter'

In [155]:
df_flights.head()

,fl_date,air_time,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,cancelled,diverted,month,day_of_week,day_of_month,year,dep_hour,arr_hour,seasons
0,2019-09-25,58.0,WN,WN,WN,954,WN,N232WN,954,14193,PNS,"Pensacola, FL",10693,BNA,"Nashville, TN",1930,-8.0,2045,N,75.0,1.0,391.0,-10.0,0.0,0.0,9,2,25,2019,19,20,autum
1,2018-05-07,122.0,UA,UA,UA,740,UA,N843UA,740,11292,DEN,"Denver, CO",13495,MSY,"New Orleans, LA",0949,-5.0,1322,N,153.0,1.0,1062.0,-22.0,0.0,0.0,5,0,7,2018,09,13,spring
2,2018-07-02,287.0,AS,AS,AS,1097,AS,N853VA,1097,12264,IAD,"Washington, DC",12892,LAX,"Los Angeles, CA",1613,-8.0,1852,N,339.0,1.0,2288.0,-39.0,0.0,0.0,7,0,2,2018,16,18,summer
3,2019-07-02,35.0,AA,AA,AA,1194,AA,N934NN,1194,13303,MIA,"Miami, FL",13204,MCO,"Orlando, FL",1730,2.0,1847,N,77.0,1.0,192.0,-15.0,0.0,0.0,7,1,2,2019,17,18,summer
4,2019-06-28,55.0,WN,WN,WN,2543,WN,N7864B,2543,11540,ELP,"El Paso, TX",14107,PHX,"Phoenix, AZ",2100,-2.0,2120,N,80.0,1.0,347.0,-5.0,0.0,0.0,6,4,28,2019,21,21,summer


In [156]:
# Dividing the depature and arrival hours into day sections
df_flights['dep_hour']=df_flights['dep_hour'].astype(int)
df_flights['dep_time_during_day']='night'
df_flights.loc[df_flights['dep_hour'].between(12,18), 'dep_time_during_day']= 'afternoon'
df_flights.loc[df_flights['dep_hour'].between(19,23), 'dep_time_during_day']= 'evening'
df_flights.loc[df_flights['dep_hour'].between(4,11), 'dep_time_during_day']= 'morning'

df_flights['arr_hour']=df_flights['arr_hour'].astype(int)
df_flights['arr_time_during_day']='night'
df_flights.loc[df_flights['arr_hour'].between(12,18), 'arr_time_during_day']= 'afternoon'
df_flights.loc[df_flights['arr_hour'].between(19,23), 'arr_time_during_day']= 'evening'
df_flights.loc[df_flights['arr_hour'].between(4,11), 'arr_time_during_day']= 'morning'

In [157]:
df_flights = df_flights.drop('fl_date', axis = 1)

In [158]:
df_flights['dep_time_during_day'].value_counts()

afternoon    20640
morning      20248
evening       7949
night          185
Name: dep_time_during_day, dtype: int64

In [159]:
#Converting to air_time and arr_delay to category
def flight_duration(x):
    if x <=180:
        return 'Short'
    elif x >180 and x<360:
        return 'Medium'
    elif x>360:
        return 'Long'

df_flights['flight_duration_type']=df_flights['air_time'].apply(lambda x: flight_duration(x))
df_flights['flight_duration_type'].value_counts()

Short     42538
Medium     6318
Long        157
Name: flight_duration_type, dtype: int64

In [160]:
# Separating State name from city
df_flights['origin_state']=df_flights['origin_city_name'].apply(lambda x: x.split(', ')[1])
df_flights['destination_state']=df_flights['dest_city_name'].apply(lambda x: x.split(', ')[1])

In [161]:
# Finding the mean of departure and arrival delays based on different carriers
df_flights['dep_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['dep_delay'].transform(np.mean)
df_flights['arr_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['arr_delay'].transform(np.mean)

In [162]:
#encode arrival delay
df_flights['arr_delay_cat'] = df_flights['arr_delay'].apply(lambda x: 1 if x > 0 else 0)
# ENCODE AIRPORTS AND TAILNUM
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df_flights['mkt_carrier'] = encoder.fit_transform(df_flights[['mkt_carrier']])
df_flights['mkt_unique_carrier'] = encoder.fit_transform(df_flights[['mkt_unique_carrier']])
df_flights['mkt_carrier_fl_num'] = encoder.fit_transform(df_flights[['mkt_carrier_fl_num']])
df_flights['op_unique_carrier'] = encoder.fit_transform(df_flights[['op_unique_carrier']])
df_flights['tail_num'] = encoder.fit_transform(df_flights[['tail_num']])
df_flights['op_carrier_fl_num'] = encoder.fit_transform(df_flights[['op_carrier_fl_num']])
df_flights['origin_airport_id'] = encoder.fit_transform(df_flights[['origin_airport_id']])
df_flights['dest_airport_id'] = encoder.fit_transform(df_flights[['dest_airport_id']])

In [163]:
# Finding the air traffics based on the airports
df_flights['origin_airport_taffic']=df_flights.groupby(['origin_airport_id'])['flights'].transform(np.sum)
df_flights['dest_airport_traffic']=df_flights.groupby(['dest_airport_id'])['flights'].transform(np.sum)

In [164]:
# Converting categorical data by getting dummies


dummy_durtypes=pd.get_dummies(df_flights[['flight_duration_type', 'dep_time_during_day', 'arr_time_during_day', 'seasons']])

df_flights=pd.concat([df_flights, dummy_durtypes], axis=1).drop(['flight_duration_type', 'dep_time_during_day', 'arr_time_during_day', 'seasons'], axis=1)


In [167]:
df_flights.isnull().sum()

air_time                         0
mkt_unique_carrier               0
branded_code_share               0
mkt_carrier                      0
mkt_carrier_fl_num               0
op_unique_carrier                0
tail_num                         0
op_carrier_fl_num                0
origin_airport_id                0
origin                           0
origin_city_name                 0
dest_airport_id                  0
dest                             0
dest_city_name                   0
crs_dep_time                     0
dep_delay                        0
crs_arr_time                     0
dup                              0
crs_elapsed_time                 0
flights                          0
distance                         0
arr_delay                        0
cancelled                        0
diverted                         0
month                            0
day_of_week                      0
day_of_month                     0
year                             0
dep_hour            

In [168]:
df_flights=df_flights.drop(['dep_delay','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
                            'mkt_carrier_fl_num', 'op_unique_carrier','op_carrier_fl_num', 
                            'origin', 'origin_city_name','dup','dest','dest_city_name',
                            'origin_state', 'destination_state', 
                            'cancelled','diverted',
                           ], axis=1)

In [169]:
df_flights.head()

,air_time,tail_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay,month,day_of_week,day_of_month,year,dep_hour,arr_hour,dep_delay_mean_by_carrier,arr_delay_mean_by_carrier,arr_delay_cat,origin_airport_taffic,dest_airport_traffic,flight_duration_type_Long,flight_duration_type_Medium,flight_duration_type_Short,dep_time_during_day_afternoon,dep_time_during_day_evening,dep_time_during_day_morning,dep_time_during_day_night,arr_time_during_day_afternoon,arr_time_during_day_evening,arr_time_during_day_morning,arr_time_during_day_night,seasons_autum,seasons_spring,seasons_summer
0,58.0,920.0,274.0,44.0,1930,2045,75.0,1.0,391.0,-10.0,9,2,25,2019,19,20,10.407439,3.269226,0,72.0,493.0,0,0,1,0,1,0,0,0,1,0,0,1,0,0
1,122.0,4481.0,92.0,240.0,0949,1322,153.0,1.0,1062.0,-22.0,5,0,7,2018,9,13,11.105513,6.464257,0,1732.0,331.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
2,287.0,4581.0,163.0,193.0,1613,1852,339.0,1.0,2288.0,-39.0,7,0,2,2018,16,18,3.696758,0.513032,0,560.0,1511.0,0,1,0,1,0,0,0,1,0,0,0,0,0,1
3,35.0,5470.0,225.0,215.0,1730,1847,77.0,1.0,192.0,-15.0,7,1,2,2019,17,18,10.396965,5.962407,0,555.0,892.0,0,0,1,1,0,0,0,1,0,0,0,0,0,1
4,55.0,4038.0,111.0,267.0,2100,2120,80.0,1.0,347.0,-5.0,6,4,28,2019,21,21,10.407439,3.269226,0,120.0,1103.0,0,0,1,0,1,0,0,0,1,0,0,0,0,1


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

* PCA testing

In [58]:
'''from sklearn.decomposition import PCA
import matplotlib.pyplot as plt'''

'from sklearn.decomposition import PCA\nimport matplotlib.pyplot as plt'

### Modeling

Use different ML techniques to predict each problem.

- linear
- Naive Bayes
- Random Forest Regressor
- SVM classification
- XGBoost regresspr
- The ensemble of your own choice

#### pickle module to save model

In [37]:
import pickle

* target and features

In [17]:
y = np.array(df_flights.arr_delay)

In [18]:
y_cat = np.array(df_flights.arr_delay_cat)

In [19]:
X_df = df_flights.drop(['arr_delay', 'arr_delay_cat'], axis = 1)

In [20]:
X = np.array(X_df)

* train test split and making samples

In [21]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.70,test_size=0.30, random_state=101, shuffle = True)

In [22]:
X_train_sample = pd.DataFrame(X_train).sample(frac = 0.1).values

In [23]:
#creating y_train_cat and y_test_cat
X_train, X_test, y_train_cat, y_test_cat = model_selection.train_test_split(X, y_cat, train_size=0.70,test_size=0.30, random_state=101, shuffle = True)

In [24]:
y_train_sample = pd.DataFrame(y_train).sample(frac = 0.1).values

In [25]:
y_trainCat_sample = pd.DataFrame(y_train_cat).sample(frac = 0.1).values

In [26]:
X_test_sample = pd.DataFrame(X_test).sample(frac = 0.1).values

* Scaling

In [70]:
'''scaler = StandardScaler()
X = scaler.fit_transform(X)'''

'scaler = StandardScaler()\nX = scaler.fit_transform(X)'

In [71]:
'''y = (y - y.mean()) / y.std()'''

'y = (y - y.mean()) / y.std()'

In [27]:
from sklearn.metrics import r2_score

* Scaling PIPE

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [29]:
pipe = Pipeline([('scaler', StandardScaler())])
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler())])

#### Linear

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [31]:
#logistic regression
linear_model = LinearRegression()

In [32]:
linear_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('linear_model', LinearRegression())])

In [33]:
linear_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_model', LinearRegression())])

In [34]:
y_pred_linear = linear_pipe.predict(X_test)

In [35]:
r2_score = linear_pipe.score(X_test, y_test)

In [36]:
r2_score

0.03316525333461695

* pickling

In [38]:
with open('model_linear_pickle', 'wb') as linear_file:
    pickle.dump(linear_pipe, linear_file)

In [39]:
with open('model_linear_pickle', 'rb') as linear_file:
    model_linear = pickle.load(linear_file)

### Linear_ElasticNet

In [40]:
from sklearn.linear_model import ElasticNet
elasnet_model = ElasticNet()

In [41]:
hyperparameters ={
    'elasnet_model__alpha': [1],
    'elasnet_model__l1_ratio': [0.0001]
}

In [42]:
elasnet_model_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('elasnet_model', ElasticNet())])
elasnet_model_grid = GridSearchCV(estimator=elasnet_model_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [43]:
'''elasnet_model_grid.fit(X_train_sample, y_trainCat_sample.ravel())
y_pred_randFor_class = elasnet_model_grid.predict(X_test_sample)'''

'elasnet_model_grid.fit(X_train_sample, y_trainCat_sample.ravel())\ny_pred_randFor_class = elasnet_model_grid.predict(X_test_sample)'

In [44]:
'''elasnet_model_grid.best_estimator_'''

'elasnet_model_grid.best_estimator_'

* adjusted to best hyperparameters

In [45]:
elasnet_model_grid.fit(X_train, y_train_cat.ravel())

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('elasnet_model', ElasticNet())]),
             param_grid={'elasnet_model__alpha': [1],
                         'elasnet_model__l1_ratio': [0.0001]},
             scoring='r2')

In [46]:
y_pred_randFor_class = elasnet_model_grid.predict(X_test)

In [47]:
elasnet_model_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('elasnet_model', ElasticNet(alpha=1, l1_ratio=0.0001))])

In [48]:
r2_score = elasnet_model_grid.score(X_test, y_test_cat)
r2_score

0.023124280459166813

### Polyfeats

In [49]:
from sklearn.preprocessing import PolynomialFeatures

In [50]:
poly_model = PolynomialFeatures()

In [51]:
poly_model_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('poly_model', PolynomialFeatures(2)), ('linear_reg', LinearRegression())])

* test with sample to find best hyperparameters

In [52]:
poly_model_pipe.fit(X_train_sample, y_train_sample.ravel())
y_pred_poly = poly_model_pipe.predict(X_test_sample)

In [53]:
poly_model_pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('poly_model', PolynomialFeatures()),
                ('linear_reg', LinearRegression())])

In [54]:
r2_score = poly_model_pipe.score(X_test, y_test_cat)
r2_score

-4.887142585456985e+18

#### Naives Bayes, GaussianNB Naive Bayes

In [55]:
from sklearn.naive_bayes import GaussianNB

In [56]:
NB_Gauss_model = GaussianNB()

In [57]:
NB_Gauss_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('NB_Gauss_model', GaussianNB())])

In [58]:
NB_Gauss_pipe.fit(X_train, y_train_cat)

Pipeline(steps=[('scaler', StandardScaler()), ('NB_Gauss_model', GaussianNB())])

In [59]:
y_pred_NBGauss = NB_Gauss_pipe.predict(X_test)

In [60]:
r2_score = NB_Gauss_pipe.score(X_test, y_test_cat)
r2_score

0.6013610071452875

* pickling

In [61]:
with open('model_Gaussian_pickle', 'wb') as Gaussian_file:
    pickle.dump(NB_Gauss_pipe, Gaussian_file)

In [62]:
with open('model_Gaussian_pickle', 'rb') as Gaussian_file:
    model_Gaussian = pickle.load(Gaussian_file)

#### Random Forest Regressor

In [71]:
from sklearn.ensemble import RandomForestRegressor

In [72]:
random_forest_regression_model = RandomForestRegressor()

In [73]:
hyperparameters = {
    'rand_forrestReg__n_estimators': [1000],
    'rand_forrestReg__max_depth': [2],
    'rand_forrestReg__min_samples_split':[6], 
    'rand_forrestReg__bootstrap':[True],
    'rand_forrestReg__criterion' :['mse']
}

In [78]:
Forest_reg_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('rand_forrestReg', RandomForestRegressor())])
forest_reg_grid = GridSearchCV(estimator=Forest_reg_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [79]:
'''forest_reg_grid.fit(X_train_sample, y_train_sample.ravel())
y_pred_randFor_class = forest_reg_grid.predict(X_test_sample)'''

'forest_reg_grid.fit(X_train_sample, y_train_sample.ravel())\ny_pred_randFor_class = forest_reg_grid.predict(X_test_sample)'

In [83]:
#forest_reg_grid.best_estimator_

* adjusted to best hyperparameters

In [81]:
forest_reg_grid.fit(X_train, y_train)
y_pred_randFo = forest_reg_grid.predict(X_test)

In [82]:
forest_reg_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rand_forrestReg',
                 RandomForestRegressor(max_depth=2, min_samples_split=6,
                                       n_estimators=1000))])

In [84]:
r2_score = forest_reg_grid.score(X_test, y_test)

In [85]:
r2_score

0.011335594586944109

* pickling

In [86]:
with open('model_Forrest_reg_pickle', 'wb') as forest_reg_file:
    pickle.dump(forest_reg_grid, forest_reg_file)

In [87]:
with open('model_Forrest_reg_pickle', 'rb') as forest_reg_file:
    model_Forrest_reg_pickle = pickle.load(forest_reg_file)

#### SVM classification

In [88]:
from sklearn.svm import SVC, SVR, LinearSVC

In [89]:
model_svm_class = SVC()

In [90]:
hyperparameters = {'model_svm_class__kernel': ['linear'],
                   'model_svm_class__C':[1],
                   'model_svm_class__degree':[1]
}
SVC_grid_class = GridSearchCV(estimator=model_svm_class, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

In [91]:
svm_class_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('model_svm_class', SVC())])
smv_class_grid = GridSearchCV(estimator=svm_class_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [92]:
smv_class_grid.fit(X_train_sample, y_trainCat_sample.ravel())
y_pred_svm = smv_class_grid.predict(X_test_sample)

In [93]:
smv_class_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model_svm_class', SVC(C=1, degree=1, kernel='linear'))])

* adjusted to best hyperparameters

In [94]:
smv_class_grid.fit(X_train,y_train_cat)
y_pred_svc_cat = smv_class_grid.predict(X_test)

In [95]:
smv_class_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model_svm_class', SVC(C=1, degree=1, kernel='linear'))])

In [96]:
r2_score = smv_class_grid.score(X_test, y_test_cat)

In [97]:
r2_score

-0.4397323298434088

* pickling

In [98]:
with open('model_smv_class_pickle', 'wb') as smv_class_file:
    pickle.dump(smv_class_grid, smv_class_file)

In [99]:
with open('model_smv_class_pickle', 'rb') as smv_class_file:
    model_smv_class_pickle = pickle.load(smv_class_file)

#### XGBoost regressor

In [100]:
import xgboost as xgb

In [101]:
xg_reg = xgb.XGBRegressor()

In [102]:
hyperparameters = {
    'xg_reg__objective' : ['reg:squarederror'],
    'xg_reg__colsample_bytree':[0.1],
    'xg_reg__n_estimators': [3000],
    'xg_reg__max_depth': [4],
    'xg_reg__learning_rate': [0.0001],
    'xg_reg__alpha': [5]
}

In [103]:
svm_class_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('xg_reg', xgb.XGBRegressor())])
xgb_grid = GridSearchCV(estimator=svm_class_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [104]:
xgb_grid.fit(X_train_sample, y_train_sample.ravel())
y_pred_xbg = xgb_grid.predict(X_test_sample)

In [105]:
xgb_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xg_reg',
                 XGBRegressor(alpha=5, base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.0001,
                              max_delta_step=0, max_depth=4, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=3000, n_jobs=4, num_parallel_tree=1,
                              random_state=0, reg_alpha=5, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

* adjusted to best hyperparameters

In [106]:
xgb_grid.fit(X_train,y_train)
y_pred_xbg = xgb_grid.predict(X_test)

In [107]:
xgb_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xg_reg',
                 XGBRegressor(alpha=5, base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.0001,
                              max_delta_step=0, max_depth=4, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=3000, n_jobs=4, num_parallel_tree=1,
                              random_state=0, reg_alpha=5, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [108]:
r2_score = xgb_grid.score(X_test, y_test)
r2_score

-0.0031786326725999903

* pickling

In [109]:
with open('model_xgb_grid_pickle', 'wb') as xgb_reg_file:
    pickle.dump(xgb_grid, xgb_reg_file)

In [110]:
with open('model_xgb_grid_pickle', 'rb') as xgb_reg_file:
    model_xgb_grid_pickle = pickle.load(xgb_reg_file)

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.